In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

In [ ]:
images = tfds.load(name="bsds500", split="validation") \
                .map(datasets.get_image) \
                .map(datasets.to_float32) \
                .map(datasets.from_255_to_1_range)
kernels = tfds.load(name='schelten_kernels/dmsp', split='test') \
                .map(datasets.crop_kernel_to_size) \
                .map(datasets.get_value('kernel')) \
                .map(datasets.to_float32) \
                .map(dppp.conv2D_filter_rgb)

In [ ]:
# Specify the level of noise added to the blurred image
noise_stddev = 0.01

In [ ]:
# Pick a new random image
image = datasets.get_one_example(images, random=True)[None,...]
kernel = datasets.get_one_example(kernels, random=True)
degraded = dppp.blur(image, kernel, noise_stddev, clip_final=False)

In [ ]:
# Load the denoiser
denoiser, (_, _) = dppp.load_denoiser('models/drunet+_0.0-0.2.h5')

In [ ]:
denoiser_stddevs = [0.05, 0.125, 0.2]
num_steps = 100

# Print the PSNR every 20th step
callbacks = [dppp.callback_print_psnr('psnr', 20, image)]

reconstructed = []

for dstddev in denoiser_stddevs:
    print()
    print(f"Running for denoiser stddev {dstddev:0.2f}:")
    print("=================================")
    reconstructed.append(dppp.dmsp_deblur_nb(degraded, kernel, noise_stddev,
                                        denoiser, dstddev,
                                        num_steps=num_steps, callbacks=callbacks))
    

print()
print("FINAL RESULTS:")
for dstddev, rec in zip(denoiser_stddevs, reconstructed):
    psnr = dppp.psnr(image, rec).numpy()[0]
    print(f"PSNR for denoiser stddev {dstddev:0.2f}: {psnr}")